## First importing the libraries

In [1]:
!pip install geopy

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


## 1. Download and Explore Dataset

In [2]:
import requests
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page

<Response [200]>

In [3]:
#check if upload is succesfull
page.status_code

200

In [4]:
#import Beautiful Soup
!pip install beautifulsoup4
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
html = list(soup.children)[2]

In [6]:
body = list(html.children)[3]

In [7]:
table=soup.find('table')
table

<table cellpadding="2" cellspacing="0" rules="all" style="width:100%; border-collapse:collapse; border:1px solid #ccc;">
<tbody><tr>
<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M1A</b><br/><span style="font-size:85%;"><i>Not assigned</i></span>
</p>
</td>
<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M2A</b><br/><span style="font-size:85%;"><i>Not assigned</i></span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M3A</b><br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>)</span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M4A</b><br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>)</span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M5A</b><br/><span style="font-size:85%;"><a hr

In [8]:
#create a table 
table_contents=[]

#looks for a table on the website and extracts the text from that table (bt looking for the tags 'td'). Skip the 'not assigned'
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
df=pd.DataFrame(table_contents)

In [9]:
df.shape

(103, 3)

In [10]:
df

Borough  \
0                                           North York   
1                                           North York   
2                                     Downtown Toronto   
3                                           North York   
4                                         Queen's Park   
5                                            Etobicoke   
6                                          Scarborough   
7                                           North York   
8                                            East York   
9                                     Downtown Toronto   
10                                          North York   
11                                           Etobicoke   
12                                         Scarborough   
13                                          North York   
14                                           East York   
15                                    Downtown Toronto   
16                                                York   
17                                           Etobicoke   
18                                         Scarborough   
19                                        East Toronto   
20                                    Downtown Toronto   
21                                                York   
22                                         Scarborough   
23                                           East York   
24                                    Downtown Toronto   
25                                    Downtown Toronto   
26                                         Scarborough   
27                                          North York   
28                                          North York   
29                                           East York   
30                                    Downtown Toronto   
31                                        West Toronto   
32                                         Scarborough   
33                                          North York   
34                                          North York   
35                               East YorkEast Toronto   
36                                    Downtown Toronto   
37                                        West Toronto   
38                                         Scarborough   
39                                          North York   
40                                          North York   
41                                        East Toronto   
42                                    Downtown Toronto   
43                                        West Toronto   
44                                         Scarborough   
45                                          North York   
46                                          North York   
47                                        East Toronto   
48                                    Downtown Toronto   
49                                          North York   
50                                          North York   
51                                         Scarborough   
52                                          North York   
53                                          North York   
54                                        East Toronto   
55                                          North York   
56                                                York   
57                                          North York   
58                                         Scarborough   
59                                          North York   
60                                          North York   
61                                     Central Toronto   
62                                     Central Toronto   
63                                                York   
64                                                York   
65                                         Scarborough   
66                                          North York   
67                                     Central Toronto   
68                                     Centra

In [11]:
# Repalce the strange values.
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

Borough  \
0                North York   
1                North York   
2          Downtown Toronto   
3                North York   
4              Queen's Park   
5                 Etobicoke   
6               Scarborough   
7                North York   
8                 East York   
9          Downtown Toronto   
10               North York   
11                Etobicoke   
12              Scarborough   
13               North York   
14                East York   
15         Downtown Toronto   
16                     York   
17                Etobicoke   
18              Scarborough   
19             East Toronto   
20         Downtown Toronto   
21                     York   
22              Scarborough   
23                East York   
24         Downtown Toronto   
25         Downtown Toronto   
26              Scarborough   
27               North York   
28               North York   
29                East York   
30         Downtown Toronto   
31             West Toronto   
32              Scarborough   
33               North York   
34               North York   
35   East York/East Toronto   
36         Downtown Toronto   
37             West Toronto   
38              Scarborough   
39               North York   
40               North York   
41             East Toronto   
42         Downtown Toronto   
43             West Toronto   
44              Scarborough   
45               North York   
46               North York   
47             East Toronto   
48         Downtown Toronto   
49               North York   
50               North York   
51              Scarborough   
52               North York   
53               North York   
54             East Toronto   
55               North York   
56                     York   
57               North York   
58              Scarborough   
59               North York   
60               North York   
61          Central Toronto   
62          Central Toronto   
63                     York   
64                     York   
65              Scarborough   
66               North York   
67          Central Toronto   
68          Central Toronto   
69             West Toronto   
70                Etobicoke   
71              Scarborough   
72               North York   
73          Central Toronto   
74          Central Toronto   
75             West Toronto   
76              Mississauga   
77                Etobicoke   
78              Scarborough   
79          Central Toronto   
80         Downtown Toronto   
81             West Toronto   
82              Scarborough   
83          Central Toronto   
84         Downtown Toronto   
85              Scarborough   
86          Central Toronto   
87         Downtown Toronto   
88                Etobicoke   
89                Etobicoke   
90              Scarborough   
91         Downtown Toronto   
92   Downtown Toronto Stn A   
93                Etobicoke   
94      Etobicoke Northwest   
95              Scarborough   
96         Downtown Toronto   
97         Downtown Toronto   
98                Etobicoke   
99         Downtown Toronto   
100   East Toronto Business   
101               Etobicoke   
102               Etobicoke   

                                          Neighborhood PostalCode  
0                                            Parkwoods        M3A  
1                                     Victoria Village        M4A  
2                            Regent Park, Harbourfront        M5A  
3                     Lawrence Manor, Lawrence Heights        M6A  
4                        Ontario Provincial Government        M7A  
5                                     Islington Avenue        M9A  
6                                       Malvern, Rouge        M1B  
7                                      Don Mills North        M3B  
8                      Parkview Hill, Woodbine Gardens        M4B  
9                             Garden District, Ryerson        M5B  
10                                           Gl

In [12]:
# Sorting the columns so that the match the example in the assignment
df=df[['PostalCode', 'Borough','Neighborhood']]
df

PostalCode                 Borough  \
0          M3A              North York   
1          M4A              North York   
2          M5A        Downtown Toronto   
3          M6A              North York   
4          M7A            Queen's Park   
5          M9A               Etobicoke   
6          M1B             Scarborough   
7          M3B              North York   
8          M4B               East York   
9          M5B        Downtown Toronto   
10         M6B              North York   
11         M9B               Etobicoke   
12         M1C             Scarborough   
13         M3C              North York   
14         M4C               East York   
15         M5C        Downtown Toronto   
16         M6C                    York   
17         M9C               Etobicoke   
18         M1E             Scarborough   
19         M4E            East Toronto   
20         M5E        Downtown Toronto   
21         M6E                    York   
22         M1G             Scarborough   
23         M4G               East York   
24         M5G        Downtown Toronto   
25         M6G        Downtown Toronto   
26         M1H             Scarborough   
27         M2H              North York   
28         M3H              North York   
29         M4H               East York   
30         M5H        Downtown Toronto   
31         M6H            West Toronto   
32         M1J             Scarborough   
33         M2J              North York   
34         M3J              North York   
35         M4J  East York/East Toronto   
36         M5J        Downtown Toronto   
37         M6J            West Toronto   
38         M1K             Scarborough   
39         M2K              North York   
40         M3K              North York   
41         M4K            East Toronto   
42         M5K        Downtown Toronto   
43         M6K            West Toronto   
44         M1L             Scarborough   
45         M2L              North York   
46         M3L              North York   
47         M4L            East Toronto   
48         M5L        Downtown Toronto   
49         M6L              North York   
50         M9L              North York   
51         M1M             Scarborough   
52         M2M              North York   
53         M3M              North York   
54         M4M            East Toronto   
55         M5M              North York   
56         M6M                    York   
57         M9M              North York   
58         M1N             Scarborough   
59         M2N              North York   
60         M3N              North York   
61         M4N         Central Toronto   
62         M5N         Central Toronto   
63         M6N                    York   
64         M9N                    York   
65         M1P             Scarborough   
66         M2P              North York   
67         M4P         Central Toronto   
68         M5P         Central Toronto   
69         M6P            West Toronto   
70         M9P               Etobicoke   
71         M1R             Scarborough   
72         M2R              North York   
73         M4R         Central Toronto   
74         M5R         Central Toronto   
75         M6R            West Toronto   
76         M7R             Mississauga   
77         M9R               Etobicoke   
78         M1S             Scarborough   
79         M4S         Central Toronto   
80         M5S        Downtown Toronto   
81         M6S            West Toronto   
82         M1T             Scarborough   
83         M4T         Central Toronto   
84         M5T        Downtown Toronto   
85         M1V             Scarborough   
86         M4V         Central Toronto   
87         M5V        Downtown Toronto   
88         M8V               Etobicoke   
89         M9V               Etobicoke   
90         M1W             Scarborough   
91         M4W        Downtown Toronto   
92         M5W  Downtown Toronto Stn A   
93         M8W               Etobicoke   
94         M9W

In [13]:
df.shape

(103, 3)